In [ ]:
import serial
import time
import mysql.connector
import pandas as pd

In [ ]:
arduino = serial.Serial(
    port='COM6',
    baudrate=115200,
    )

In [ ]:
mysql = mysql.connector.connect(
    host="localhost",
    database = "exercicio_php",
    port=3306,
    user="root",
    password="1234")

In [ ]:
def readSerial():
    message = ''
    while (arduino.inWaiting() > 0):
        message += arduino.readline(1).decode()
    
    if ("nodeNumber" in message):
        print(message)
        
        nodeNumber = getNodeNumber(message)
        nodeStatus = getNodeStatus(message)
        
        if (nodeNumber != ''):
        
            dados = pd.read_sql(
            sql = "SELECT * FROM mesh_network WHERE node = " + str(nodeNumber) + " ",
            con = mysql
            )

            if dados.empty:
                sql = "insert into mesh_network VALUES( " + str(nodeNumber) + ", " + str(nodeStatus) + " )"
                print("insert")
            else:
                sql = "update mesh_network SET state = " + str(nodeStatus) + " WHERE node = " + str(nodeNumber) +" "
                print("update")

            mycursor = mysql.cursor()
            mycursor.execute(sql)
            mysql.commit()

In [ ]:
def getNodeNumber(message):
    node_index = message.find("nodeNumber")
    starting_index = node_index + 12
    final_index = starting_index + 1

    nodeNumber = message[starting_index:final_index]

    return nodeNumber

In [ ]:
def getNodeStatus(message):
    node_index = message.find("state")
    starting_index = node_index + 7
    final_index = starting_index + 1

    state = message[starting_index:final_index]

    if (state == 't'):
        state = 1
    else:
        state = 0

    return state

In [ ]:
while (True):
    readSerial()
    time.sleep(0.01)